In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium import FeatureGroup
from config import vworld_key
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [2]:
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])

In [3]:
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

In [4]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

### 청주시_도시재생계획구역

In [8]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/27.청주시_도시재생계획구역.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
crp_df = pd.json_normalize(geojson_data) # city_revitalize_planning_map_df
crp_df['geometry'] = crp_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)

# 폴리곤의 중점을 찾음
centroid = crp_df['geometry'].iloc[0].centroid.buffer(0.015) # 1도의 위도 변화는 대략 111.32 킬로미터
crp_df['centroid_polygon_geometry'] = [centroid]

### 격자(매핑용)

In [11]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/26.청주시_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))

In [12]:
# crp_df에서 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
polygon_expanded = crp_df['centroid_polygon_geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')
#grid_map_df['geometry'] = GeoSeries(grid_map_df['geometry'])

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid = grid_map_df[grid_map_df['geometry'].intersects(polygon)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택, within 은 포함
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid_expanded = grid_map_df[grid_map_df['geometry'].intersects(polygon_expanded)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택

### 청주시 버스정류장 현황

In [14]:
busstop_df = pd.read_csv('SBJ_2309_001/10.청주시_버스정류장현황.csv')
busstop_df = geo_transform(busstop_df)
busstop_df

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,bis_id,stn_nm,lon,lat,geometry
0,4391461,기점,127.47443,36.64312,POINT (127.47443 36.64312)
1,4391471,회차점,127.47763,36.64462,POINT (127.47763 36.64462)
2,4391481,종점,127.47637,36.64430,POINT (127.47637 36.64430)
3,4389171,청주국제공항,127.49548,36.72235,POINT (127.49548 36.72235)
4,4373991,청주공항역,127.49132,36.72117,POINT (127.49132 36.72117)
...,...,...,...,...,...
3209,4303441,성산1리,127.42052,36.74148,POINT (127.42052 36.74148)
3210,4303341,옥산산업단지제2승강장,127.37507,36.69898,POINT (127.37507 36.69898)
3211,4303361,성재세월교,127.33785,36.71890,POINT (127.33785 36.71890)
3212,4303281,문덕리.다리골,127.52280,36.46963,POINT (127.52280 36.46963)


In [ ]:
# map 생성
m = folium.Map(location=[36.627797, 127.511943],  zoom_start=12)

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

for _, row in busstop_df.iterrows() :
    popup_text = f"{row['bis_id']} - {row['stn_nm']}"
    folium.Circle(location=(row['lat'], row['lon']), radius=5, color='#FF580B',
            fill='#FF580B').add_to(m).add_child(folium.Popup(popup_text, max_width=200))
    
m

### 청주시 상세 도로망 시각화

In [59]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/13.청주시_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : LineString(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

In [60]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0].centroid.buffer(0.05) # buffer로 원도심 centoid에서 약 5km반경 영역

# 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadsystem = roadsystem_df[roadsystem_df['geometry'].within(polygon)].reset_index(drop=True)

In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_roadsystem.iterrows():
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 3}
    ).add_to(m)
    
m

#### 청주시 추정교통량

In [73]:
# 도로 id와 geometry를 매핑한 딕셔너리 생성 : 추정교통량 data와 매핑 예정
road_geo_dic = dict(zip(roadsystem_df['properties.link_id'], roadsystem_df['geometry']))

In [74]:
traffic_df = pd.read_csv('SBJ_2309_001/12.청주시_추정교통량.csv')
traffic_df.rename(columns={'year' : '연도'}, inplace=True)
traffic_df['연도'] = traffic_df['연도'].astype(str)
traffic_df['link_id'] = traffic_df['link_id'].astype(str)
traffic_df['link_id'] = traffic_df['link_id'].apply(lambda x : x[:-2])
traffic_df

C:\Users\user\AppData\Local\Temp\ipykernel_25512\3309607067.py:1: DtypeWarning:

Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.



,연도,link_id,road_rank,road_length,road_nm,sido_nm,sgg_nm,emd_nm,timeslot,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT
0,2019,900000055,107,0.327926,NaN,충청북도,청주시 서원구,현도면,all,1198,136,24,1038
1,2019,900000055,107,0.327926,NaN,충청북도,청주시 서원구,현도면,all,1531,126,24,1381
2,2019,900000070,107,0.341045,NaN,충청북도,청주시 서원구,현도면,all,1076,126,24,925
3,2019,900000070,107,0.341045,NaN,충청북도,청주시 서원구,현도면,all,1268,119,25,1124
4,2019,564304590,101,0.253755,당진영덕고속도로,충청북도,청주시 상당구,문의면,all,2457,351,85,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340495,2021,563413568,108,0.256000,경부고속도로,충청북도,청주시 흥덕구,옥산면,9,234,210,8,17
1340496,2021,563413564,108,0.249000,경부고속도로,충청북도,청주시 흥덕구,옥산면,9,165,97,13,54
1340497,2021,563506287,108,0.926000,경부고속도로,충청북도,청주시 흥덕구,옥산면,9,503,354,14,135
1340498,2021,563506038,108,1.456000,경부고속도로,충청북도,청주시 흥덕구,옥산면,9,498,96,14,387


In [99]:
# 딕셔너리 생성 및 연도 & DataFrame 매핑
traffic_df_year_dic = {}
year_list = ['2019', '2020', '2021']

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0].centroid.buffer(0.05) # buffer로 원도심 centoid에서 약 5km반경 영역

for year in year_list:
    traffic_df_year = traffic_df[traffic_df['연도'] == year].copy() # 특정 연도로만 필터링
    traffic_df_year['geometry'] = traffic_df_year['link_id'].map(road_geo_dic)
    # polygon을 사용하여 폴리곤 내의 도로망 필터링
    traffic_df_year = gpd.GeoDataFrame(traffic_df_year, geometry='geometry')
    traffic_df_year = traffic_df_year[traffic_df_year['geometry'].within(polygon)].reset_index(drop=True)
    traffic_df_year = traffic_df_year[traffic_df_year['timeslot'] == 'all']
    traffic_df_year_dic[year] = traffic_df_year

## 이거 도로 id 로 필터링하면 여러개 나오는데 이거 합쳐야 할듯...?

In [101]:
traffic_df_year_dic['2019'][traffic_df_year_dic['2019']['link_id'] == '564409449']

,연도,link_id,road_rank,road_length,road_nm,sido_nm,sgg_nm,emd_nm,timeslot,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT,geometry
0,2019,564409449,103,0.01185,단재로,충청북도,청주시 상당구,금천동,all,12566,9272,882,2412,"LINESTRING (127.49373 36.62460, 127.49366 36.6..."
1,2019,564409449,103,0.01185,단재로,충청북도,청주시 상당구,금천동,all,13677,9909,1075,2693,"LINESTRING (127.49373 36.62460, 127.49366 36.6..."


#### 2020년도 유동인구 HEATMAP 시계열 변화(시간대별)

In [ ]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=12)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = filtered_floating_population_day['2020']['lat']
longitudes = filtered_floating_population_day['2020']['lon']

# 시간대별 히트맵 데이터 생성
heat_data = []
for day in filtered_floating_population_day['2020'].columns[1:-2]:
    volume = filtered_floating_population_day['2020'][day]
    heat_data_day = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_day)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[day for day in filtered_floating_population_day['2020'].columns[1:-2]], auto_play=False, max_opacity=0.6, radius=18).add_to(m)

# 지도 출력
m

#### 2021년도 유동인구 HEATMAP 시계열 변화(시간대별)

In [ ]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=12)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = filtered_floating_population_day['2021']['lat']
longitudes = filtered_floating_population_day['2021']['lon']

# 시간대별 히트맵 데이터 생성
heat_data = []
for day in filtered_floating_population_day['2021'].columns[1:-2]:
    volume = filtered_floating_population_day['2021'][day]
    heat_data_day = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_day)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[day for day in filtered_floating_population_day['2020'].columns[1:-2]], auto_play=False, max_opacity=0.6, radius=18).add_to(m)

# 지도 출력
m

#### 2022년도 유동인구 HEATMAP 시계열 변화(시간대별)

In [ ]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=12)

# 데이터프레임에서 위도, 경도, 시간대별 데이터 추출
latitudes = filtered_floating_population_day['2022']['lat']
longitudes = filtered_floating_population_day['2022']['lon']

# 시간대별 히트맵 데이터 생성
heat_data = []
for day in filtered_floating_population_day['2022'].columns[1:-2]:
    volume = filtered_floating_population_day['2022'][day]
    heat_data_day = [[lat, lon, vol] for lat, lon, vol in zip(latitudes, longitudes, volume)]
    heat_data.append(heat_data_day)

# HeatMapWithTime을 추가
plugins.HeatMapWithTime(heat_data, index=[day for day in filtered_floating_population_day['2020'].columns[1:-2]], auto_play=False, max_opacity=0.6, radius=18).add_to(m)

# 지도 출력
m

#### 격자에 매핑 & 평일과 주말을 통합한 격자의 유동인구 시각화

In [42]:
floating_population_df_day

,연도,mon,tue,wed,thu,fri,sat,sun,lon,lat,geometry
94061,202001,49.28,51.12,50.75,54.67,52.78,51.24,46.99,127.483602,36.631043,POINT (127.48360 36.63104)
94062,202001,112.92,114.62,115.31,121.40,121.01,123.04,114.80,127.483602,36.631494,POINT (127.48360 36.63149)
94063,202001,119.92,122.71,120.15,129.35,125.74,112.61,104.84,127.483602,36.631944,POINT (127.48360 36.63194)
94064,202001,82.00,85.41,84.07,90.94,87.51,82.13,71.36,127.483602,36.632395,POINT (127.48360 36.63240)
94366,202001,59.56,63.09,60.48,64.91,63.13,57.99,50.31,127.484162,36.630592,POINT (127.48416 36.63059)
...,...,...,...,...,...,...,...,...,...,...,...
4687534,202212,73.23,72.03,70.74,72.24,79.62,56.35,45.40,127.492551,36.631945,POINT (127.49255 36.63195)
4687535,202212,284.77,279.83,278.79,281.96,304.53,289.08,239.32,127.492550,36.632396,POINT (127.49255 36.63240)
4687536,202212,101.30,100.28,98.78,101.39,103.76,83.35,71.96,127.492550,36.632847,POINT (127.49255 36.63285)
4687958,202212,263.03,263.19,253.53,263.11,280.09,290.16,267.59,127.493110,36.628339,POINT (127.49311 36.62834)


In [78]:
# 최종저장용 df 정의 -> 각 지점들을 grid폴리곤에 매핑시켜 grid별 유동인구로 변환 -> 
filtered_floating_population_totday = pd.DataFrame()
for grid in tqdm(filtered_grid['geometry'].unique()):
    # 격자와 매핑하기 위해 격자 내에 있는 지점으로만 필터링
    subset = floating_population_df_day[floating_population_df_day['geometry'].within(grid)]
    # 해당 구역 grid df 설정(접합용) 
    grid_df = pd.DataFrame({'geometry' : [grid]})
    # 중간저장용 df 정의
    concat_df = pd.DataFrame()
    
    for date in floating_population_df_day['연도'].unique():
        date_df = pd.DataFrame({'연도' : [date[:4]]}) # 해당 연도 df 설정(접합용) 
        subset_date = subset[subset['연도'] == date]
        # 열 별로 더하기 및 df 변환
        column_sums = subset_date.iloc[:,1:8].sum()
        column_sums_df = pd.DataFrame(column_sums).transpose()
        # 최종 df 접합 및 filtered_floating_population_totday에 concat
        column_sums_df = pd.concat([date_df, column_sums_df], axis=1)
        concat_df = pd.concat([concat_df, column_sums_df])
    else:
        # 각 연도 별로 평균계산 및 df 변환 -> 월별 유동인구를 평균내었기 때문에, 최종값은 해당 격자의 월별 평균 유동인구가 됨.
        column_means_2020 = concat_df[concat_df['연도'].str.startswith('2020')].iloc[:,1:8].mean()
        date_df_2020 = pd.DataFrame({'연도' : ['2020']})
        column_means_2020_df = pd.DataFrame(column_means_2020).transpose()
        column_means_2020_df = pd.concat([column_means_2020_df.iloc[:,:5].mean(axis=1), column_means_2020_df.iloc[:,5:].mean(axis=1)], axis=1)
        column_means_2020_df.columns = ['평일_유동인구', '주말_유동인구']
        tot_df_2020 = pd.concat([date_df_2020, column_means_2020_df, grid_df], axis=1)
        
        column_means_2021 = concat_df[concat_df['연도'].str.startswith('2021')].iloc[:,1:8].mean()
        date_df_2021 = pd.DataFrame({'연도' : ['2021']})
        column_means_2021_df = pd.DataFrame(column_means_2021).transpose()
        column_means_2021_df = pd.concat([column_means_2021_df.iloc[:,:5].mean(axis=1), column_means_2021_df.iloc[:,5:].mean(axis=1)], axis=1)
        column_means_2021_df.columns = ['평일_유동인구', '주말_유동인구']
        tot_df_2021 = pd.concat([date_df_2021, column_means_2021_df, grid_df], axis=1)
        
        column_means_2022 = concat_df[concat_df['연도'].str.startswith('2022')].iloc[:,1:8].mean()
        date_df_2022 = pd.DataFrame({'연도' : ['2022']})
        column_means_2022_df = pd.DataFrame(column_means_2022).transpose()
        column_means_2022_df = pd.concat([column_means_2022_df.iloc[:,:5].mean(axis=1), column_means_2022_df.iloc[:,5:].mean(axis=1)], axis=1)
        column_means_2022_df.columns = ['평일_유동인구', '주말_유동인구']
        tot_df_2022 = pd.concat([date_df_2022, column_means_2022_df, grid_df], axis=1)
        
        tot_df = pd.concat([tot_df_2020, tot_df_2021, tot_df_2022])
        filtered_floating_population_totday = pd.concat([filtered_floating_population_totday, tot_df])

filtered_floating_population_totday

100%|██████████| 156/156 [00:22<00:00,  7.06it/s]


,연도,평일_유동인구,주말_유동인구,geometry
0,2020,116.320333,98.387917,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2021,91.805833,73.114167,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2022,108.369000,89.299167,"POLYGON ((127.4921705012675 36.62987958832503,..."
0,2020,1767.508500,1412.525417,POLYGON ((127.48881291670402 36.64430329023720...
0,2021,2133.972333,1679.542500,POLYGON ((127.48881291670402 36.64430329023720...
...,...,...,...,...
0,2021,165.902500,66.894583,POLYGON ((127.49105137605241 36.63528850233756...
0,2022,217.775500,95.224167,POLYGON ((127.49105137605241 36.63528850233756...
0,2020,54.346333,27.015000,POLYGON ((127.48433936092893 36.63799230366292...
0,2021,45.899667,22.871667,POLYGON ((127.48433936092893 36.63799230366292...


In [ ]:
# 특정 value값에 따라 색상을 지정하는 함수
def rate_color(val, dataframe):
    lst = dataframe['평일_유동인구'].tolist() + dataframe['주말_유동인구'].tolist()
    sorted_lst = sorted(lst, reverse=True)
    top_25 = int(0.25 * len(sorted_lst)) # 상위 25%
    top_50 = int(0.50 * len(sorted_lst)) # 상위 50%
    top_75 = int(0.75 * len(sorted_lst)) # 상위 75%
    if val > top_25:
        return 'red'
    elif top_25 > val >= top_50:
        return 'orange'
    elif top_50 > val >= top_75:
        return 'yellow'
    elif top_75 > val >= 0:
        return 'green'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "> &nbsp; 유동인구 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 상위 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 상위 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 상위 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 상위 100%
     </div>
     """

# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_ = folium.FeatureGroup(name="2020년", overlay=False)
m_weekday = folium.FeatureGroup(name="평일 유동인구", overlay=False)
m_weekend = folium.FeatureGroup(name="주말 유동인구", overlay=False)

week_lst = ['평일', '주말']
layers_lst = [m_weekday, m_weekend]
layers_dic = dict(zip(week_lst, layers_lst))

# 연도 데이터프레임을 만들고 평일 & 주말 레이어 추가
filtered_floating_population_totday_2020 = filtered_floating_population_totday[filtered_floating_population_totday['연도'] == '2020']

# grid 값을 기준으로 그룹화
for grid in filtered_floating_population_totday_2020['geometry'].unique():
    subset = filtered_floating_population_totday_2020[filtered_floating_population_totday_2020['geometry'] == grid].copy()
    weekday_val = subset['평일_유동인구'].iloc[0]
    color = rate_color(weekday_val, filtered_floating_population_totday_2020)
    popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{weekday_val}</div>"
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(layers_dic['평일']).add_child(folium.Popup(popup_text), name=str(year))
    
    weekend_val = subset['주말_유동인구'].iloc[0]
    color = rate_color(weekend_val, filtered_floating_population_totday_2020)
    popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{weekend_val}</div>"
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(layers_dic['주말']).add_child(folium.Popup(popup_text), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 주중 & 주말 선택할 수 있게 구성
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m

In [ ]:
# 특정 value값에 따라 색상을 지정하는 함수
def rate_color(val, dataframe):
    lst = dataframe['평일_유동인구'].tolist() + dataframe['주말_유동인구'].tolist()
    sorted_lst = sorted(lst, reverse=True)
    top_25 = int(0.25 * len(sorted_lst)) # 상위 25%
    top_50 = int(0.50 * len(sorted_lst)) # 상위 50%
    top_75 = int(0.75 * len(sorted_lst)) # 상위 75%
    if val > top_25:
        return 'red'
    elif top_25 > val >= top_50:
        return 'orange'
    elif top_50 > val >= top_75:
        return 'yellow'
    elif top_75 > val >= 0:
        return 'green'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "> &nbsp; 유동인구 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 상위 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 상위 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 상위 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 상위 100%
     </div>
     """

# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_ = folium.FeatureGroup(name="2021년", overlay=False)
m_weekday = folium.FeatureGroup(name="평일 유동인구", overlay=False)
m_weekend = folium.FeatureGroup(name="주말 유동인구", overlay=False)

week_lst = ['평일', '주말']
layers_lst = [m_weekday, m_weekend]
layers_dic = dict(zip(week_lst, layers_lst))

# 연도 데이터프레임을 만들고 평일 & 주말 레이어 추가
filtered_floating_population_totday_2021 = filtered_floating_population_totday[filtered_floating_population_totday['연도'] == '2021']

# grid 값을 기준으로 그룹화
for grid in filtered_floating_population_totday_2021['geometry'].unique():
    subset = filtered_floating_population_totday_2021[filtered_floating_population_totday_2021['geometry'] == grid].copy()
    weekday_val = subset['평일_유동인구'].iloc[0]
    color = rate_color(weekday_val, filtered_floating_population_totday_2021)
    popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{weekday_val}</div>"
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(layers_dic['평일']).add_child(folium.Popup(popup_text), name=str(year))
    
    weekend_val = subset['주말_유동인구'].iloc[0]
    color = rate_color(weekend_val, filtered_floating_population_totday_2021)
    popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{weekend_val}</div>"
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(layers_dic['주말']).add_child(folium.Popup(popup_text), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 주중 & 주말 선택할 수 있게 구성
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m

In [ ]:
# 특정 value값에 따라 색상을 지정하는 함수
def rate_color(val, dataframe):
    lst = dataframe['평일_유동인구'].tolist() + dataframe['주말_유동인구'].tolist()
    sorted_lst = sorted(lst, reverse=True)
    top_25 = int(0.25 * len(sorted_lst)) # 상위 25%
    top_50 = int(0.50 * len(sorted_lst)) # 상위 50%
    top_75 = int(0.75 * len(sorted_lst)) # 상위 75%
    if val > top_25:
        return 'red'
    elif top_25 > val >= top_50:
        return 'orange'
    elif top_50 > val >= top_75:
        return 'yellow'
    elif top_75 > val >= 0:
        return 'green'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "> &nbsp; 유동인구 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 상위 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 상위 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 상위 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 상위 100%
     </div>
     """

# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_ = folium.FeatureGroup(name="2022년", overlay=False)
m_weekday = folium.FeatureGroup(name="평일 유동인구", overlay=False)
m_weekend = folium.FeatureGroup(name="주말 유동인구", overlay=False)

week_lst = ['평일', '주말']
layers_lst = [m_weekday, m_weekend]
layers_dic = dict(zip(week_lst, layers_lst))

# 연도 데이터프레임을 만들고 평일 & 주말 레이어 추가
filtered_floating_population_totday_2022 = filtered_floating_population_totday[filtered_floating_population_totday['연도'] == '2022']

# grid 값을 기준으로 그룹화
for grid in filtered_floating_population_totday_2022['geometry'].unique():
    subset = filtered_floating_population_totday_2022[filtered_floating_population_totday_2022['geometry'] == grid].copy()
    weekday_val = subset['평일_유동인구'].iloc[0]
    color = rate_color(weekday_val, filtered_floating_population_totday_2022)
    popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{weekday_val}</div>"
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(layers_dic['평일']).add_child(folium.Popup(popup_text), name=str(year))
    
    weekend_val = subset['주말_유동인구'].iloc[0]
    color = rate_color(weekend_val, filtered_floating_population_totday_2022)
    popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{weekend_val}</div>"
    folium.GeoJson(
        subset['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(layers_dic['주말']).add_child(folium.Popup(popup_text), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 주중 & 주말 선택할 수 있게 구성
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m